## Problem Set 3

**Total points**: 24

**Background on two data sets**: here, we're going to use two datasets to practice regex patterns and merging/matching. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers, with more details on the policy background available at last spring's course page under the "Social impact practicum context" header: https://rebeccajohnson88.github.io/qss20/docs/sip_finalproject.html


The following datasets are located in `pset3_inputdata` (need to unzip): 

- `jobs`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


In [262]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Regex and exact matching (10 points)



# 1.1 Load data on debarments and job postings (0 points)

Load the following datasets stored in `pset3_inputdata`
    
- Historical H2A debarments (debar.csv); call this `debar`
- Q1 2021 H2A job postings (jobs.csv); call this `jobs`


In [263]:
# your code here to load the data 
debar = pd.read_csv("debar.csv")
jobs = pd.read_csv("jobs.csv")

### 1.2 Try exact merge on business name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, subset to the following columns and print the rows with exact matches:

-`Employer_NAME` and `Name` 
- Date range of debarment (`Start date` and `End date` in `debar`)
- Location from each data (`City, State` in `debar` and `EMPLOYER_CITY` and `EMPLOYER_STATE` in `jobs`)

**Concepts tested and resources**: part A tests using `pd.merge` for exact merging; part B tests column subsetting 
   - Slides on exact merging: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/w22_slides/05_qss20_w22_unit5_mergingexact.pdf 
   - Code with examples of exact merging: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/03_merging_exact_solutions.ipynb 
   - Code with example of column subsetting: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/00_pandas_datacleaning_solutions.ipynb


In [264]:
# your code here to do the merge and printing of exact match(es)
merged_on_biz_name = pd.merge(jobs, debar, left_on='EMPLOYER_NAME', right_on='Name')
exact_matches = merged_on_biz_name[['EMPLOYER_NAME', 'Name', 'Start date', 'End date', 'City, State', 'EMPLOYER_CITY', 'EMPLOYER_STATE']]
exact_matches

,EMPLOYER_NAME,Name,Start date,End date,"City, State",EMPLOYER_CITY,EMPLOYER_STATE
0,Rafael Barajas,Rafael Barajas,9/23/2016,9/22/2017,"Sebring, Florida",Port St. Lucie,FL


## 1.3 Targeted regex

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 1.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 

**Resources**:
    - Activity code with list comprehension: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/00_pandas_datacleaning_solutions.ipynb 
    - Sampling from a list without replacement using the `random` module: https://note.nkmk.me/en/python-random-choice-sample-choices/ 
    

In [265]:
## insert your code to turn into uppercase here
EMPLOYER_NAME_UPPER = [name.upper() for name in jobs['EMPLOYER_NAME']]
Name_Upper = [name.upper() for name in debar['Name']]

In [266]:
## insert your code for the random sample
random.sample(EMPLOYER_NAME_UPPER, 15)
random.sample(Name_Upper, 15)

['RALPH & BEVERLY FISHER',
 'SAUCEDA CONTRACTORS, INC',
 'SCOTT & COMPANY FARM, L.L.C.',
 'PURE BEAUTY FARMS INC',
 'THE GROWERS COMPANY, INC.',
 'KALIR ENTERPRISES INC',
 'TEMP.LABOR, LLC',
 'JOHN OR CASSANDRA WEST',
 'MCF4 SOLUTIONS, LLC',
 'WATKINS FAMILY PARTNERSHIP',
 'MITCH BENSON FARMS',
 'WESLEY BLADOW',
 'JUBILEE FARMS, INC',
 'MERCER RANCHES, INC.',
 'SCHLESSIGER FARMS LLC']

['COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*',
 'XAVIER HORNE',
 'J & L FARMS',
 'LEONARD SMITH FARMS',
 'OMEGA LAMB, LLC',
 'J&J HARVESTING',
 'DONNA LUCIO',
 'QUENTIN SCHELL',
 'MAPLE RIDGE CUSTOM SERVICES, LLC',
 'AGECY I LLC',
 'YESENIA PEREZ',
 'PROMAX INC.',
 'JOHN & NETA LEOPKY FARMS',
 'TURNER FARMS',
 'CHRIS ROBINSON']

In [267]:
## insert your code for assigning the uppercase names back to the data
jobs['EMPLOYER_NAME'] = EMPLOYER_NAME_UPPER
debar['Name'] = Name_Upper

### 1.3.2 Cleaning up punctuation (6 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content

**Resources**:
    - Regex slides: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/w22_slides/06_qss20_w22_unit6_regex.pdf 
    - Regex activity code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/04_basicregex_solutions.ipynb

In [268]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [269]:
## insert your code here with the regex pattern for part A
regex = r'(INC|LLC|CO)\.'

## insert your code to use re.sub to apply the pattern to the test cases for part B
re.sub(regex, r'\1' , pos_example_1)
re.sub(regex, r'\1' , pos_example_2)
re.sub(regex, r'\1' , neg_example)

'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

C. Use that pattern in conjunction with `re.sub` and list comprehension to clean the columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables

**Concepts and resources**: for the last part of row subsetting and printing two columns in each dataframe, use the `.loc` examples shown here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/00_pandas_datacleaning_solutions.ipynb


In [270]:
## your code here to clean the columns
jobs['name_clean'] = [re.sub(regex, r'\1', name) for name in jobs['EMPLOYER_NAME']]
debar['name_clean'] = [re.sub(regex, r'\1', name) for name in debar['Name']]

In [271]:
## your code here to print the head
jobs_changed_names = jobs[['EMPLOYER_NAME', 'name_clean']].loc[jobs['EMPLOYER_NAME']!=jobs['name_clean']]
debar_changed_names = debar[['Name', 'name_clean']].loc[debar['Name']!=debar['name_clean']]

debar_changed_names.head()
jobs_changed_names.head()

,Name,name_clean
3,ANTON FERTILIZER INC.,ANTON FERTILIZER INC
4,"GREAT PLAINS FLUID SERVICE, INC.","GREAT PLAINS FLUID SERVICE, INC"
5,PROMAX INC.,PROMAX INC
13,REIMER'S INC.,REIMER'S INC
19,CISCO PRODUCE INC.,CISCO PRODUCE INC


,EMPLOYER_NAME,name_clean
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC.","FARM LABOR ASSOCIATION FOR GROWERS, INC"
14,"MCLAIN FARMS, INC.","MCLAIN FARMS, INC"
17,"BONNIE PLANTS, INC.","BONNIE PLANTS, INC"
18,"B & W QUALITY GROWERS, INC.","B & W QUALITY GROWERS, INC"


### Optional extra credit (1.3.3) regex to separate companies from individuals (4 points)

You notice some employers in `debar` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [272]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern
company_regex = r'(.*)\s\(COMPANY\)\sAND\s(.*)\s\(INDIVIDUAL\)'

## your code here to apply it to the pos_example
re.findall(company_regex, pos_example)

## your code here to apply it to the negative example
re.findall(company_regex, neg_example)

[('COUNTY FAIR FARM', 'ANDREW WILLIAMSON')]

[]

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [273]:
## your code here to create these columns
debar[['co_name', 'ind_name']] = [[(re.findall(company_regex, name)[0][0]), (re.findall(company_regex, name)[0][1])] if re.findall(company_regex, name) 
                    else ['NaN', 'NaN']
                    for name in debar['name_clean']] 
                                  
debar.loc[debar['name_clean']=="COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"][['co_name', 'ind_name']]

,co_name,ind_name
108,COUNTY FAIR FARM,ANDREW WILLIAMSON


   
D. Print three columns for the rows in `debar` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [274]:
# your code here to print these columns
debar.loc[(debar['name_clean']=="COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*") | 
          (debar['name_clean']=="CISCO PRODUCE INC")][['name_clean', 'co_name', 'ind_name', 'Violation']]

,name_clean,co_name,ind_name,Violation
19,CISCO PRODUCE INC,NaN,NaN,Failure to respond to audit (no response)
56,CISCO PRODUCE INC,NaN,NaN,Impeding the Audit Process – Non- Response
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM,ANDREW WILLIAMSON,WHD Debarment


# 2 Fuzzy matching to match debarments to jobs (14 points)


## 2.1 Preprocessing state names (2 points)

You want to block on state but notice that states in `debar` have a mix of two digit codes and full state names (e.g., GA versus Georgia) while states in `jobs` are all two-digit state codes

A. Run the code below to load the `states` crosswalk (this matches state abbreviations with their full name). 

B. Use that crosswalk to create a field in `debar` that has the two-digit state abbreviation for all locations (hint: you may need to first split the `City, State` string on the ", " or use str.replace to extract the state into a new column before converting it to all abbreviations)

**Hint**: the GitHub issue contains the value_counts() for the cleaned two-digit states after this step

C. Use an `assert` statement to check that `all` the states in `debar` are two-digits after the cleaning

**Notes**: you can filter out states that are NaN

**Concepts and resources**:

- For part B, draw on regex or pandas str functions to create a `State` variable from the `City, State` column in `debar`
- Then, one approach is to use `pd.merge` to merge the state crosswalk onto debar (example head() shown in the GitHub issue)
- You may want to structure the `pd.merge` to only merge on rows *without* two-digit names to the crosswalk with those two-digit names and `pd.concat` to bind the dataframe back together
- How to structure an assert statement: https://www.w3schools.com/python/ref_keyword_assert.asp

In [275]:
## code to load state crosswalk
## source- https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c
cw_location = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/'
cw_filename = 'state_codes.html'
states = pd.read_html(cw_location + cw_filename)[0]

In [276]:
## your code here to add two-digit state codes
# debar['City, State']
regex2 = r'(.*,\s)([A-Za-z ]+)'
debar['State'] = [re.sub(regex2, r'\2', state) for state in debar['City, State'].astype(str)]
# debar['State']
long_only = debar.loc[(debar['State'].str.len()>2) & (debar['State'].isnull()==False)]
short_only = debar.loc[(debar['State'].str.len()==2) & (debar['State'].isnull()==False)]
states_merged = pd.merge(long_only, states, left_on='State', right_on='Description')
states_merged["State"] = states_merged['Code']
to_concat = [short_only, states_merged]
final = pd.concat(to_concat)

for state in final['State']:
    assert len(state) == 2


## 2.2 step by step fuzzy matching (4 points)



A. Write fuzzy matching code (don't yet put inside a user-defined function, you'll do that in 2.3) that:

- Blocks on two-digit state code
- Finds matches based on similarity between the employer name (`name_clean`) in `debar` (uppercase and cleaned) and `name_clean` in `jobs` (uppercase and cleaned). You can choose which distance metric and threshold to use (feel free to set a threshold low enough to get some matches even if that leads to some false positives).

For the steps after compute, just take any match with non-zero value rather than using a classifier (so skip the k-means or e-m step from the class example code)

**Concepts and resources**:

- Solutions code here (more consolidated): https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/05_merging_fuzzy_activity_solutions.ipynb
- Example code here (more step by step): https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/05_merging_fuzzy_codeexample.ipynb 

**Hint**: you may need to deduplicate records in the datasets for the recordlinkage package to work. See drop_duplicates within pandas; the `subset` command within `drop_duplicates` allows you to only consider certain columns for duplicates; drop based on duplicates in the two-digit state code and `name_clean`. After you drop_duplicates, you may need to use reset_index to create a new index in the data, eg:

`dedup_df = df.drop_duplicates(subset = ['state', 'name_clean']).reset_index(drop = True)`

`dedup_df['df_ind'] = dedup_df.index`

In [277]:
## your fuzzy matching code here
newcolumns = {'EMPLOYER_STATE': 'State'}
jobs = jobs.rename(columns = newcolumns, inplace = False)

dedup_jobs = jobs.drop_duplicates(subset = ['State', 'name_clean']).reset_index(drop = True)
dedup_jobs['jobs_ind'] = dedup_jobs.index
dedup_debar = final.drop_duplicates(subset = ['State', 'name_clean']).reset_index(drop = True)
dedup_debar['debar_ind'] = dedup_debar.index

matcher = recordlinkage.Index()
matcher.block('State')

links = matcher.index(dedup_debar, dedup_jobs)

compare = recordlinkage.Compare()
threshold = 0.75
compare.string('name_clean', 'name_clean', method='jaro', threshold=threshold)

compare_vectors = compare.compute(links, dedup_debar, dedup_jobs)

compare_vectors_df = pd.DataFrame(compare_vectors.reset_index())
compare_vectors_df.columns = ['debar_ind', 'jobs_ind', 'name_match']

no_zero = compare_vectors_df.loc[compare_vectors_df['name_match']!=0]


add_debar = pd.merge(no_zero, dedup_debar, how='left', on='debar_ind')


both = pd.merge(add_debar, dedup_jobs, how='left', on='jobs_ind', suffixes=['_debar', '_jobs'])


<Index>

<Compare>


B. Print the following columns in the resulting matched dataset and comment on examples of ones that seem like true positive matches and ones that seem like false positive matches:

- `name_clean` for jobs and debar
- `state` in debar
- `state` in jobs

In [278]:
# your code here
both[['name_clean_jobs', 'name_clean_debar', 'State_debar', 'State_jobs']]

,name_clean_jobs,name_clean_debar,State_debar,State_jobs
0,NELSON FARMS,OLSON FARMS,ND,ND
1,RSK FARMS,OLSON FARMS,ND,ND
2,SORUM FARMS,OLSON FARMS,ND,ND
3,MARVIN OPP FARMS,GERMAN FARMS,ND,ND
4,MORGAN CRAFT,GERMAN FARMS,ND,ND
5,TESSMAN FARMS,GERMAN FARMS,ND,ND
6,RYAN KADRMAS,GERMAN FARMS,ND,ND
7,HAMEL FARM,GERMAN FARMS,ND,ND
8,SORUM FARMS,GERMAN FARMS,ND,ND
9,SPANIER FARMS,GERMAN FARMS,ND,ND


There are a few matches that that we think could be potentially true positive matches. Slash EV Ranch is probably the same as Slash E.V. Ranch LLC. Avoyelles Crawfish & Seafood LLC could be the same company as Avoyelles Honey Co LLC. Dixie Belle, Inc and Dixie Bell #2 are probably the same. Most of the rest are probably false positives. For example: German Farms got matched to many different other companies. So did F&W Farms. Most of these are very unlikely to be true positive matches.

## 2.3 Put the code from 2.2 into a fuzzy matching function that generalizes some steps (8 points)

You want to see how the matches change if you add the city and not only state as a field and also want to automate the process of matching a bit to try different distance thresholds.

A. Extract the City from the `City, State` column of `debar`

B. Convert that new `city` column to uppercase and convert the `EMPLOYER_CITY` column in `jobs` to uppercase

C. Write a function surrounding the code in `recordlinkage` that you wrote in problem 2.2 (so you don't need to recode the package from scratch) that (1) takes in each dataset, (2) blocks on two-digit state, and (3) fuzzy matches on employer name and employer city. See below notes on partial versus full credit for function structure. 

D. Execute the function with a couple different string distance thresholds and print the resulting matches for each

6 out of 8 points: function takes arguments for input datasets, varname to block on, two varnames to fuzzy match on (`name_clean` and `city)`, string distance function, and string distance threshold
    
8 out of 8: above but function is also general enough that it takes a variable # of strings to match on--- so should work if you either execute just using employer name or also work if you execute using employer name and employer city as the fuzzy variables.

**Concepts and resources**: same as above. One hint is that for us, the easiest way to get to the full credit solution was to feed the function a dictionary where each key corresponds to one string variable to fuzzy match on; the values contain that variable's name in each dataset, the distance metric, and the distance threshold.

In [279]:
## your code here to create the city variable
city_regex = r'(.*)(,\s)(.*)'
dedup_debar['city'] = [re.sub(city_regex, r'\1', pair) for pair in dedup_debar['City, State']]

dedup_debar['city_upper'] = dedup_debar['city'].str.upper()

dedup_jobs['city_upper'] = dedup_jobs['EMPLOYER_CITY'].str.upper()

In [280]:
def fuzzy(df1, df2, block, match_dict, func, thres):
    matcher = recordlinkage.Index()
    matcher.block(block)
    links = matcher.index(df1, df2)
    
    compare = recordlinkage.Compare()
    [compare.string(key, match_dict[key], method=func, threshold=thres) for key in match_dict]
    
    compare_vectors = compare.compute(links, df1, df2)
    compare_vectors_df = pd.DataFrame(compare_vectors.reset_index())
    
#     compare_vectors_df.columns = ["df1_ind", "jf2_ind", 'match1', ma]
    compare_vectors_df['sum'] = compare_vectors_df.iloc[:, 2:].sum(axis=1)
    only_match = compare_vectors_df.loc[compare_vectors_df['sum']>=2]
    only_match

    
    df1['df1_ind'] = df1.index
    df2['df2_ind'] = df2.index
    add_df1 = pd.merge(only_match, df1, how='left', left_on='level_0', right_on='df1_ind')
    both = pd.merge(add_df1, df2, how='left', left_on='level_1', right_on='df2_ind', suffixes=['_df1', '_df2'])
    return both

In [281]:
## your code here to execute it with a couple different string distance thresholds
# dedup_debar.columns
# dedup_jobs.columns
dict = {'name_clean': 'name_clean',
       'city_upper': 'city_upper'}
test1 = fuzzy(dedup_debar, dedup_jobs, 'State', dict, "jaro", 0.5)
test1[['name_clean_df1', 'name_clean_df2', 'city_upper_df1', 'city_upper_df2', 'State_df1', 'State_df2']]
test2 = fuzzy(dedup_debar, dedup_jobs, 'State', dict, "jaro", 0.6)
test2[['name_clean_df1', 'name_clean_df2', 'city_upper_df1', 'city_upper_df2', 'State_df1', 'State_df2']]
test3 = fuzzy(dedup_debar, dedup_jobs, 'State', dict, "jaro", 0.65)
test3[['name_clean_df1', 'name_clean_df2', 'city_upper_df1', 'city_upper_df2', 'State_df1', 'State_df2']]
test4 = fuzzy(dedup_debar, dedup_jobs, 'State', dict, "jaro", 0.7)
test4[['name_clean_df1', 'name_clean_df2', 'city_upper_df1', 'city_upper_df2', 'State_df1', 'State_df2']]


,name_clean_df1,name_clean_df2,city_upper_df1,city_upper_df2,State_df1,State_df2
0,J&J HARVESTING,JEROME K. FOLVAG,LEADS,WILLISTON,ND,ND
1,J&J HARVESTING,CHAD & JENNIFER OLSON,LEADS,COLEHARBOR,ND,ND
2,J&J HARVESTING,ROBERT LANDEIS,LEADS,PETERSBURG,ND,ND
3,J&J HARVESTING,JORDAN PELTON,LEADS,HALLIDAY,ND,ND
4,J&J HARVESTING,MELIN BROTHERS FARM,LEADS,KENMARE,ND,ND
...,...,...,...,...,...,...
2550,DEAL FAMILY FARM,"SWEET BERRY FARMS, LLC",FRANKLIN,IVANHOE,NC,NC
2551,DEAL FAMILY FARM,"SUGAR SHACK FARMS, LLC",FRANKLIN,IVANHOE,NC,NC
2552,DEAL FAMILY FARM,"LEWIS NURSERY AND FARMS, INC",FRANKLIN,ROCKY POINT,NC,NC
2553,DEAL FAMILY FARM,HARRIS FAMILY FARMS,FRANKLIN,ENFIELD,NC,NC


,name_clean_df1,name_clean_df2,city_upper_df1,city_upper_df2,State_df1,State_df2
0,J&J HARVESTING,HARSTAD TRUCKING,LEADS,PLAZA,ND,ND
1,OLSON FARMS,B & B FARMS,DOUGLAS,MOUNTAIN,ND,ND
2,GERMAN FARMS,BRYAN KRINKE,FULLERTON,SCRANTON,ND,ND
3,GERMAN FARMS,JEROME K. FOLVAG,FULLERTON,WILLISTON,ND,ND
4,GERMAN FARMS,Z AND Z FEEDERS LLP,FULLERTON,HEBRON,ND,ND
...,...,...,...,...,...,...
187,C HANGING L RANCH LLP,THORNHILL RANCH PARTNERSHIP,CHOTEAU,ZORTMAN,MT,MT
188,C HANGING L RANCH LLP,S BAR B RANCH,CHOTEAU,CHINOOK,MT,MT
189,C HANGING L RANCH LLP,"BUYAN RANCH, INC",CHOTEAU,SHERIDAN,MT,MT
190,C HANGING L RANCH LLP,TANDE RANCH INC,CHOTEAU,SCOBEY,MT,MT


,name_clean_df1,name_clean_df2,city_upper_df1,city_upper_df2,State_df1,State_df2
0,GERMAN FARMS,SORUM FARMS,FULLERTON,FLAXTON,ND,ND
1,SRT FARMS,TODD OTAHAL FARMS LLC,MORTON,ROBSTOWN,TX,TX
2,SRT FARMS,BELDING FARMS LLC,MORTON,FORT STOCKTON,TX,TX
3,FIRST AMERICAN HOLDING,GARRETT FLYING SERVICE INC,AUBREY,DANBURY,TX,TX
4,DOVE CREEK FARMS,MOORE'S HONEY FARM,MOUNT VERNON,KOUNTZE,TX,TX
5,YOLANDA CHAVEZ FARMING,"ALCO HARVESTING, LLC",SANTA MARIA,SANTA MARIA,CA,CA
6,YOLANDA CHAVEZ FARMING,CHASEN GOAT GRAZING,SANTA MARIA,SANTA ROSA,CA,CA
7,RUBEN RUIZ (DESOTO HARVESTING),"SUN CITRUS HARVESTING, INC",ARCADIA,ARCADIA,FL,FL
8,RUBEN RUIZ (DESOTO HARVESTING),"CISNEROS HARVESTING, INC",ARCADIA,ARCADIA,FL,FL
9,ROSALVA GARCIA,MARIO A VARGAS,DADE CITY,LAKE PLACID,FL,FL


,name_clean_df1,name_clean_df2,city_upper_df1,city_upper_df2,State_df1,State_df2
0,GERMAN FARMS,SORUM FARMS,FULLERTON,FLAXTON,ND,ND
1,DAVID C. MARTINEZ,JOSE H. MARTINEZ,LYONS,LYONS,GA,GA
2,SLASH E.V. RANCH LLP,SLASH EV RANCH,RIFLE,RIFLE,CO,CO
3,E.V. RANCH LLP,LOV RANCH,RIFLE,RIFLE,CO,CO
4,"LOV RANCH CO, LLLP DBA LOV RANCH",LOV RANCH,RIFLE,RIFLE,CO,CO
5,"AVOYELLES HONEY CO, LLC",AVOYELLES CRAWFISH & SEAFOOD LLC,MOREAUVILLE,MOREAUVILLE,LA,LA


# 3. Optional extra credit (up to 6 points)

- For 2 points extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 6 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 

In [21]:
## your code here